<a href="https://colab.research.google.com/github/tatianamgar/Work_GPN/blob/main/Prepprocessing%20of%20the%20data%20for%20labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openpyxl==3.0.0

In [ ]:
df = pd.read_excel("/content/g09.xlsx")
df

,Код,Полное наименование ru,Нормативный документ
0,1073744,Вагон-платформа 4-х осная универсальная модели...,ТУ 24.05.001.132-01
1,1083210,Блок водораспределительный напорный БВРН-02-20...,ОПРОСНЫЙ ЛИСТ
2,1083211,Блок водораспределительный напорный БВРН-02-20...,ОПРОСНЫЙ ЛИСТ
3,1083968,БЛОК ДОЗИРОВАНИЯ РЕАГЕНТОВ БДР-ОЗНА-ДОЗАТОР-1-...,ОПРОСНЫЙ ЛИСТ
4,1085047,БЛОК ДОЗИРОВАНИЯ РЕАГЕНТОВ БДР-2/1/2,ОПРОСНЫЙ ЛИСТ
...,...,...,...
8238,980008173899,ГИДРОПРИВОД ГП-1,-
8239,980008173906,ПРУЖИНА ОСНОВНАЯ МПК 10В-75-39,ТУ ПРОИЗВОДИТЕЛЯ
8240,980008173991,Регулятор расхода МПК-10В,ТУ ПРОИЗВОДИТЕЛЯ
8241,980008705268,Мембрана 3145.000.222,ЧЕРТЕЖ 3145.000.222


In [ ]:
ntd = df['Нормативный документ'] .value_counts()

In [ ]:
ntd.to_excel("ntd.xlsx")

In [ ]:
data = df[["Полное наименование ru"]]
data = data.rename(columns={'Полное наименование ru':"text"})
data = data.dropna(subset=['text'])

In [ ]:
import gensim
import gensim.downloader as api

In [ ]:
# Create the tagged document needed for Doc2Vec
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
train_data = list(create_tagged_document(data))
print(train_data[:1])

[TaggedDocument(words='text', tags=[0])]


In [ ]:
# Init the Doc2Vec model
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
# Build the Volabulary
model.build_vocab(train_data)
# Train the Doc2Vec model
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
print(model.infer_vector(['БобышкаБП', '1', ' М', '20Х1,5(17ГС)']))

[-9.0705594e-03  6.4599188e-03  9.3177417e-03  3.8285102e-03
 -5.4775150e-03  6.4540775e-03  8.5979719e-03  5.7223947e-03
  9.7521059e-03 -5.8826036e-03 -5.6333467e-03  3.0546129e-04
 -8.1269592e-03  7.0354925e-03  4.7868686e-03 -1.0465818e-03
 -1.1005597e-03  2.4339340e-03  8.5132131e-03 -8.9769941e-03
 -6.4721461e-03  4.4712499e-03 -9.4511252e-06  6.1965408e-03
  6.1334665e-03 -1.4356803e-03  5.3297998e-03 -8.2028359e-03
  6.2181321e-03 -7.8128250e-03  3.7188784e-03  4.6065841e-03
  7.6870932e-03  2.6736439e-03  2.2319097e-03  4.1340990e-03
  5.3271314e-04 -6.4658346e-03  8.8039003e-03  2.9810872e-03
 -4.2359098e-03 -8.3244760e-03 -5.0227698e-03 -8.5605532e-03
 -1.1408483e-03 -8.7543288e-03 -2.6847403e-03  4.2281612e-05
  2.9002348e-04  7.8156432e-03]


In [ ]:
data['vector'] = data['text'].apply(lambda x: model.infer_vector(x.split(' ')))

In [ ]:
data[['text', 'vector']]

,text,vector
0,Вагон-платформа 4-х осная универсальная модели...,"[-0.0038907116, -0.008592305, -0.0008693999, -..."
1,Блок водораспределительный напорный БВРН-02-20...,"[0.004745201, -0.0012303871, -0.0069597797, 0...."
2,Блок водораспределительный напорный БВРН-02-20...,"[0.005166156, 0.0033368443, -0.0059685176, -0...."
3,БЛОК ДОЗИРОВАНИЯ РЕАГЕНТОВ БДР-ОЗНА-ДОЗАТОР-1-...,"[-0.008337457, 0.008845418, 0.006042708, -0.00..."
4,БЛОК ДОЗИРОВАНИЯ РЕАГЕНТОВ БДР-2/1/2,"[0.009648183, 0.009783902, -0.00012783495, -0...."
...,...,...
8238,ГИДРОПРИВОД ГП-1,"[-0.0066952626, -0.0050855097, -0.0046117837, ..."
8239,ПРУЖИНА ОСНОВНАЯ МПК 10В-75-39,"[0.0037205748, 0.008921426, -0.0036352302, 0.0..."
8240,Регулятор расхода МПК-10В,"[-0.0065668374, -0.00023193726, 0.009310104, -..."
8241,Мембрана 3145.000.222,"[-0.00034676542, -0.001739458, 0.0036624784, 0..."


ОТБОР ПРИМЕРОВ ДЛЯ РАЗМЕТКИ

In [ ]:
from scipy.spatial import distance

In [ ]:
# чтобы посчитать время поиска одной строки
import time


uniq_vecs = [
    {
    'index': data.index[0],
    'vector': data['vector'].tolist()[0],
    'text': data['text'].tolist()[0]
    } #создаем хранилище для отобранных примеров
] 
vec_distances = {} #записываем, чтобы запомнить расстояние между векторами

print(data['text'].tolist()[0])
for i in range(0, 150): #запускаем цикл на 150 примеров
    tic = time.perf_counter() #запускаем таймер
    
    max_dist = 0 #сюда записываем максимальную дистанцию между векторами
    candidate = {
        'index': data.index[0],
        'vector': data['vector'].tolist()[0],
        'text': data['text'].tolist()[0]
    }
    for ind, item in data.iterrows():
        vec = {
            'index': ind,
            'vector': item['vector'],
            'text': item['text']
        }
        if ind in vec_distances:
            vec_distances[ind] += distance.cosine(vec['vector'], uniq_vecs[-1]['vector'])
        else:
            vec_distances[ind] = distance.cosine(vec['vector'], uniq_vecs[0]['vector'])
        
        if vec_distances[ind] > max_dist:
            max_dist = vec_distances[ind]
            candidate = vec
    uniq_vecs.append(candidate)
    toc = time.perf_counter() #выключаем таймер
    print(candidate['text'], f"Вычисление заняло {toc - tic:0.2f} секунд")


In [ ]:
uniq_df = pd.DataFrame(uniq_vecs)
ready_data = uniq_df['text']

In [ ]:
ready_data.to_csv(path_or_buf="ready_g09.csv",sep=",", encoding = "utf-8")